In [1]:
{-# LANGUAGE OverloadedStrings #-}

import Control.Concurrent.Async
import Data.ByteArray.Encoding (convertToBase, Base(Base16))
import Crypto.Hash (hashWith)
import Crypto.Hash.Algorithms (MD5)
import qualified Data.ByteString as B
import qualified Data.ByteString.UTF8 as BU

input = "yjdafjpo"

In [2]:
import Numeric (showHex)
import qualified Data.Attoparsec.ByteString       as A
import qualified Data.Attoparsec.ByteString.Char8 as AC
import GHC.Word (Word8)

hexBS = BU.fromString . flip showHex ""

hashInput i = B.append input (BU.fromString $ show i)

next1000 index = map hashInput [index+1..index+1000]


triple bs = case (filter (\b -> B.length b >=3) $ B.group bs) of
    []     -> Nothing
    (m:ms) -> Just $ B.head m

matching byte bs = case (filter (\b -> B.length b >= 5 && B.head b == byte) $ B.group bs) of
    []     -> False
    (x:xs) -> True

calcHash inp = convertToBase Base16 $ hashWith (undefined :: MD5) $ inp

calcQHash :: Word8 -> B.ByteString -> IO Bool
calcQHash byte inp = do
    let match = matching byte $ calcHash inp
    return match

findIndex currentIndex 64    = return (currentIndex-1)
findIndex currentIndex count = case triple (calcHash (hashInput currentIndex)) of
    Just w -> do
        matches <- or <$> mapConcurrently (calcQHash w) (next1000 currentIndex)
        if matches
            then findIndex (currentIndex+1) (count+1)
            else findIndex (currentIndex+1) count
    Nothing ->   findIndex (currentIndex+1) count

findIndex 0 0

25427

In [3]:
import           Data.Sequence (Seq, ViewL(..), viewl, (|>))
import qualified Data.Sequence as Seq

stretched 0 inp = inp
stretched n inp = stretched (n-1) $ convertToBase Base16 $ hashWith (undefined :: MD5) inp

quintuples bs = filter (\b -> B.length b >= 5) $ B.group bs

initial = map (stretched 2017 . hashInput) [0..1000]

match bs = (triple bs, quintuples bs)

matches = Seq.fromList $ map match initial

predicate word pair = let
    fives = snd pair
    match = filter (B.elem word) fives
    in not $ null match

findIndex' :: Seq (Maybe Word8, [B.ByteString]) -> Int -> Int -> Int
findIndex' hashes currentIndex 64    = currentIndex - 1
findIndex' hashes currentIndex count = case viewl hashes of
    (Just w, _) :< rest | not $ Seq.null $ Seq.filter (predicate w) rest -> let
        currentIndex' = currentIndex + 1
        newHash       = match $ stretched 2017 $ hashInput (currentIndex'+1000)
        hashes'       = rest |> newHash
        in findIndex' hashes' currentIndex' (count+1)
    _ :< rest                                                          -> let
        currentIndex' = currentIndex + 1
        newHash       = match $ stretched 2017 $ hashInput (currentIndex'+1000)
        hashes'       = rest |> newHash
        in findIndex' hashes' currentIndex' count

findIndex' matches 0 0

22045